In [1]:
from collections import namedtuple
import os
os.chdir(r"C:\Users\Praveen\deepCNNClassifier\deepCNNClassifier") # making the parent directory as jupyter notebook consuder the research folder
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories


In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self)-> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks  # all values that are defined configs.yaml availabe here
        model_check_point_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([Path(model_check_point_dir), Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config

In [7]:
# PrepareCallbacksComponent
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir) # it expects log directori path
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True # it will save the best weights only
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,  # defined it as a property
            self._create_ckpt_callbacks # defined it as a property
        ]

In [10]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config=prepare_callback_config)
    callback_list=prepare_callback.get_tb_ckpt_callbacks()
    print(callback_list)
except Exception as e:
    raise e

[<keras.callbacks.TensorBoard object at 0x0000025E4F91B040>, <keras.callbacks.ModelCheckpoint object at 0x0000025E5F59C5E0>]


In [4]:
import os
os.path.dirname('artifacts/prepare_callbacks/checkpoint_dir/model.h5')

'artifacts/prepare_callbacks/checkpoint_dir'